In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd

In [3]:
train_df = pd.read_csv('/home/joaquin/Desktop/Datos/tp2/train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df = pd.read_csv('/home/joaquin/Desktop/Datos/tp2/test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
#Procesamiento de datos
X_train, X_test, y_train, y_test = train_test_split(train_df['text'], train_df['target'], test_size=0.33,
                                                  random_state=42)
FTest = test_df['text']
test_ids = test_df['id'] 

In [23]:
#Parametros
vocab_size =10000
epochs = 10
maxlen = 40
n_words = 500
embedding_dim = 16

In [7]:
#Tokenización
tokenizer = Tokenizer(num_words = n_words, oov_token = '<OOV>')

tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
sequences_FTest = tokenizer.texts_to_sequences(FTest)


padded_train = pad_sequences(sequences_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(sequences_test, maxlen = maxlen)
padded_FTest = pad_sequences(sequences_FTest, maxlen = maxlen)

In [12]:
#modelo LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 677,249
Trainable params: 677,249
Non-trainable params: 0
_________________________________________________________________


In [14]:
#entreno
model.fit(padded_train, y_train, epochs=epochs, validation_data = (padded_test,y_test))

Epoch 1/5
160/160 [==============================] - 55s 345ms/step - loss: 0.5955 - accuracy: 0.6759 - val_loss: 0.5041 - val_accuracy: 0.7593
Epoch 2/5
160/160 [==============================] - 57s 359ms/step - loss: 0.4578 - accuracy: 0.7876 - val_loss: 0.4956 - val_accuracy: 0.7648
Epoch 3/5
160/160 [==============================] - 56s 347ms/step - loss: 0.4352 - accuracy: 0.8008 - val_loss: 0.4840 - val_accuracy: 0.7712
Epoch 4/5
160/160 [==============================] - 53s 332ms/step - loss: 0.4244 - accuracy: 0.8088 - val_loss: 0.4843 - val_accuracy: 0.7799
Epoch 5/5
160/160 [==============================] - 57s 356ms/step - loss: 0.4076 - accuracy: 0.8192 - val_loss: 0.5006 - val_accuracy: 0.7656


In [18]:
#modelo LSTM bidireccional
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 40, 64)            640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 40, 128)           66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 751,489
Trainable params: 751,489
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/10
160/160 [==============================] - 80s 500ms/step - loss: 0.4015 - accuracy: 0.8259 - val_loss: 0.5013 - val_accuracy: 0.7688
Epoch 2/10
160/160 [==============================] - 79s 492ms/step - loss: 0.3945 - accuracy: 0.8304 - val_loss: 0.4997 - val_accuracy: 0.7712
Epoch 3/10
160/160 [==============================] - 71s 443ms/step - loss: 0.3849 - accuracy: 0.8306 - val_loss: 0.5082 - val_accuracy: 0.7644
Epoch 4/10
160/160 [==============================] - 72s 447ms/step - loss: 0.3746 - accuracy: 0.8384 - val_loss: 0.5173 - val_accuracy: 0.7660
Epoch 5/10
160/160 [==============================] - 70s 438ms/step - loss: 0.3647 - accuracy: 0.8443 - val_loss: 0.5365 - val_accuracy: 0.7660
Epoch 6/10
160/160 [==============================] - 74s 466ms/step - loss: 0.3546 - accuracy: 0.8504 - val_loss: 0.5317 - val_accuracy: 0.7632
Epoch 7/10
160/160 [==============================] - 71s 445ms/step - loss: 0.3474 - accuracy: 0.8527 - val_loss: 0.5487 - val_ac

In [21]:
#modelo Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=maxlen),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 64)            640000    
_________________________________________________________________
conv1d (Conv1D)              (None, 36, 128)           41088     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 689,409
Trainable params: 689,409
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/10
160/160 [==============================] - 68s 424ms/step - loss: 0.3082 - accuracy: 0.8741 - val_loss: 0.6082 - val_accuracy: 0.7497
Epoch 2/10
160/160 [==============================] - 68s 426ms/step - loss: 0.2977 - accuracy: 0.8808 - val_loss: 0.6219 - val_accuracy: 0.7557
Epoch 3/10
160/160 [==============================] - 68s 424ms/step - loss: 0.2878 - accuracy: 0.8859 - val_loss: 0.6305 - val_accuracy: 0.7417
Epoch 4/10
160/160 [==============================] - 69s 433ms/step - loss: 0.2758 - accuracy: 0.8906 - val_loss: 0.6463 - val_accuracy: 0.7501
Epoch 5/10
160/160 [==============================] - 68s 422ms/step - loss: 0.2633 - accuracy: 0.8963 - val_loss: 0.7572 - val_accuracy: 0.7374
Epoch 6/10
160/160 [==============================] - 69s 431ms/step - loss: 0.2558 - accuracy: 0.9004 - val_loss: 0.7232 - val_accuracy: 0.7390
Epoch 7/10
160/160 [==============================] - 67s 420ms/step - loss: 0.2453 - accuracy: 0.9049 - val_loss: 0.7677 - val_ac

In [26]:
#modelo GRU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                9600      
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 390       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 169,997
Trainable params: 169,997
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/10
160/160 [==============================] - 57s 356ms/step - loss: 0.6546 - accuracy: 0.6102 - val_loss: 0.5901 - val_accuracy: 0.6908
Epoch 2/10
160/160 [==============================] - 55s 342ms/step - loss: 0.5202 - accuracy: 0.7496 - val_loss: 0.5132 - val_accuracy: 0.7453
Epoch 3/10
160/160 [==============================] - 55s 345ms/step - loss: 0.4521 - accuracy: 0.7994 - val_loss: 0.4890 - val_accuracy: 0.7712
Epoch 4/10
160/160 [==============================] - 54s 340ms/step - loss: 0.4313 - accuracy: 0.8108 - val_loss: 0.4836 - val_accuracy: 0.7768
Epoch 5/10
160/160 [==============================] - 55s 341ms/step - loss: 0.4184 - accuracy: 0.8167 - val_loss: 0.4904 - val_accuracy: 0.7716
Epoch 6/10
160/160 [==============================] - 55s 343ms/step - loss: 0.4198 - accuracy: 0.8100 - val_loss: 0.5075 - val_accuracy: 0.7648
Epoch 7/10
160/160 [==============================] - 55s 342ms/step - loss: 0.4121 - accuracy: 0.8231 - val_loss: 0.4964 - val_ac

In [ ]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds

In [ ]:
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('/home/joaquin/Desktop/Datos/tp2/Keras.csv')
df_preds.head()

Tuning: 
1-  epochs = 10
    maxlen = 50
    n_words = 500
    embedding_dim = 16
Acuracy: 0.7441

2-  epochs = 6
    maxlen = 40
    n_words = 500
    embedding_dim = 16
Accuracy : 0.7656b

3-  epochs = 5
    maxlen = 40
    n_words = 300
    embedding_dim = 16
Accuracy: 0.7437
s